# Basic Usage Guide

This notebook covers the fundamentals of the simple-backtest framework:

1. Loading data with yfinance
2. Setting up commissions
3. Running a backtest with Moving Average strategy
4. Comparing multiple strategies (Buy and Hold, DCA, Moving Average)
5. Simple parameter optimization

Let's get started!

In [ ]:
# Install the library (run this cell if using Colab or if you haven't installed the package)
!pip install --upgrade --no-cache-dir simple-backtest yfinance

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime

from simple_backtest import BacktestConfig, Backtest
from simple_backtest.strategy import MovingAverageStrategy, BuyAndHoldStrategy, DCAStrategy
from simple_backtest.optimization import GridSearchOptimizer
from simple_backtest.visualization import plot_equity_curve

## 1. Loading Data with yfinance

We'll download historical stock data for Apple (AAPL) using yfinance. The framework requires OHLCV data with a DatetimeIndex.

In [3]:
# Download data from Yahoo Finance
ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2023-12-31"

print(f"Downloading {ticker} data from {start_date} to {end_date}...")
data = yf.download(ticker, start=start_date, end=end_date, progress=False)

# Display first few rows
print(f"\nData shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")
data.head()

/var/folders/9y/pp82h2912h3781myzdncvs_r0000gp/T/ipykernel_53288/1808284023.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date, progress=False)



Data shape: (1006, 5)
Date range: 2020-01-02 00:00:00 to 2023-12-29 00:00:00


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.538506,72.598884,71.292296,71.545882,135480400
2020-01-03,71.833298,72.594063,71.608692,71.765674,146322800
2020-01-06,72.405685,72.444328,70.703020,70.954195,118387200
2020-01-07,72.065155,72.671348,71.845377,72.415345,108872000
2020-01-08,73.224411,73.526303,71.768086,71.768086,132079200


In [4]:
# Check for missing values
print("Missing values:")
print(data.isnull().sum())

# Drop any rows with missing values
data = data.dropna()
print(f"\nClean data shape: {data.shape}")

Missing values:
Price   Ticker
Close   AAPL      0
High    AAPL      0
Low     AAPL      0
Open    AAPL      0
Volume  AAPL      0
dtype: int64

Clean data shape: (1006, 5)


## 2. Setting Up Commission

The framework supports multiple commission types:
- **Percentage**: Commission as percentage of trade value (e.g., 0.1%)
- **Flat**: Fixed commission per trade (e.g., $1 per trade)
- **Tiered**: Different rates for different trade sizes

Let's start with a simple percentage commission.

In [5]:
# Configure backtest with percentage commission
config = BacktestConfig(
    initial_capital=10000.0,         # Start with $10,000
    lookback_period=50,              # Use 50 days of historical data
    commission_type="percentage",    # Percentage-based commission
    commission_value=0.001,          # 0.1% per trade
    execution_price="open",          # Execute at open price
    risk_free_rate=0.02,             # 2% annual risk-free rate
)

print("Backtest Configuration:")
print(f"  Initial Capital: ${config.initial_capital:,.2f}")
print(f"  Commission: {config.commission_value*100}% per trade")
print(f"  Lookback Period: {config.lookback_period} days")

Backtest Configuration:
  Initial Capital: $10,000.00
  Commission: 0.1% per trade
  Lookback Period: 50 days


## 3. Running a Backtest with Moving Average Strategy

The Moving Average Crossover strategy:
- **Buy** when short MA crosses above long MA (golden cross)
- **Sell** when short MA crosses below long MA (death cross)

In [6]:
# Create Moving Average strategy
ma_strategy = MovingAverageStrategy(
    short_window=10,   # 10-day moving average
    long_window=30,    # 30-day moving average
    shares=10          # Trade 10 shares at a time
)

print(f"Strategy: {ma_strategy.get_name()}")
print(f"  Short Window: {ma_strategy.short_window} days")
print(f"  Long Window: {ma_strategy.long_window} days")
print(f"  Shares per trade: {ma_strategy.shares}")

Strategy: MA_10_30
  Short Window: 10 days
  Long Window: 30 days
  Shares per trade: 10


In [7]:
# Run the backtest
backtest = Backtest(data=data, config=config)
results = backtest.run([ma_strategy])

print("Backtest completed!\n")

Running strategies: 100%|██████████| 1/1 [00:00<00:00, 13.69it/s]

Backtest completed!



In [8]:
# Display performance metrics
ma_result = results.get_strategy(ma_strategy.get_name())
metrics = ma_result.metrics

print("=" * 60)
print(f"Performance Metrics - {ma_strategy.get_name()}")
print("=" * 60)
print(f"\nReturns:")
print(f"  Total Return: {metrics['total_return']:.2f}%")
print(f"  CAGR: {metrics['cagr']:.2f}%")
print(f"\nRisk Metrics:")
print(f"  Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
print(f"  Sortino Ratio: {metrics['sortino_ratio']:.2f}")
print(f"  Max Drawdown: {metrics['max_drawdown']:.2f}%")
print(f"  Volatility: {metrics['volatility']:.2f}%")
print(f"\nTrade Statistics:")
print(f"  Total Trades: {int(metrics['total_trades'])}")
print(f"  Win Rate: {metrics['win_rate']:.2f}%")
print(f"  Profit Factor: {metrics['profit_factor']:.2f}")
print(f"\nFinal Values:")
print(f"  Final Portfolio Value: ${metrics['final_value']:,.2f}")
print(f"  Initial Capital: ${config.initial_capital:,.2f}")
print("=" * 60)

Performance Metrics - MA_10_30

Returns:
  Total Return: 7.70%
  CAGR: 1.98%

Risk Metrics:
  Sharpe Ratio: 0.01
  Sortino Ratio: 0.01
  Max Drawdown: 4.58%
  Volatility: 3.08%

Trade Statistics:
  Total Trades: 31
  Win Rate: 53.33%
  Profit Factor: 2.10

Final Values:
  Final Portfolio Value: $10,769.80
  Initial Capital: $10,000.00


In [9]:
# Visualize trading signals with buy/sell markers
fig = ma_result.plot_trades(data)
fig.show()

## 4. Comparing Multiple Strategies

Let's compare three different strategies:
1. **Moving Average Crossover** - Active trading strategy
2. **Buy and Hold** - Passive benchmark strategy
3. **Dollar Cost Averaging (DCA)** - Regular investment strategy

In [10]:
# Create multiple strategies
strategies = [
    MovingAverageStrategy(
        short_window=10,
        long_window=30,
        shares=10,
        name="MA_10_30"
    ),
    BuyAndHoldStrategy(
        shares=50,  # Buy 50 shares at the start
        name="BuyAndHold"
    ),
    DCAStrategy(
        investment_amount=500,  # Invest $500 each time
        interval_days=30,       # Every 30 days
        name="DCA_Monthly"
    )
]

print("Strategies to compare:")
for s in strategies:
    print(f"  - {s.get_name()}")

Strategies to compare:
  - MA_10_30
  - BuyAndHold
  - DCA_Monthly


In [11]:
# Run backtest with all strategies
backtest = Backtest(data=data, config=config)
results = backtest.run(strategies)

print("Backtest completed for all strategies!\n")

Backtest completed for all strategies!



In [12]:
# Compare results - specify metrics to include total_trades
comparison_df = results.compare(
    metrics=['total_return', 'cagr', 'sharpe_ratio', 'max_drawdown', 'total_trades', 'win_rate']
)
print("\nStrategy Comparison:")
print(comparison_df)


Strategy Comparison:
             total_return       cagr  sharpe_ratio  max_drawdown  \
benchmark      227.911014  36.839365      1.085038     30.601132   
DCA_Monthly     78.401088  16.518269      0.671937     29.700455   
BuyAndHold      66.820383  14.471123      0.839019     17.100523   
MA_10_30         7.698038   1.977902      0.007804      4.583326   

             total_trades    win_rate  
benchmark             1.0    0.000000  
DCA_Monthly          20.0  100.000000  
BuyAndHold            2.0  100.000000  
MA_10_30             31.0   53.333333  


In [13]:
# Visualize trading signals for all strategies
trade_figures = results.plot_trades(data)
for name, fig in trade_figures.items():
    fig.show()

In [14]:
# Find best strategy by Sharpe ratio
best_result = results.best_strategy('sharpe_ratio')
print(f"\nBest Strategy (by Sharpe Ratio): {best_result.name}")
print(f"Sharpe Ratio: {best_result.metrics['sharpe_ratio']:.2f}")


Best Strategy (by Sharpe Ratio): BuyAndHold
Sharpe Ratio: 0.84


In [15]:
# Visualize comparison
fig = results.plot_comparison()
fig.update_layout(title="Strategy Comparison - Equity Curves")
fig.show()

## 5. Parameter Optimization

Let's optimize the Moving Average strategy by testing different window combinations to find the best parameters.

In [16]:
# Define parameter space to search
param_space = {
    'short_window': [5, 10, 15, 20],      # Test different short windows
    'long_window': [30, 40, 50, 60],      # Test different long windows
    'shares': [10]                         # Keep shares constant
}

print("Parameter Space:")
for param, values in param_space.items():
    print(f"  {param}: {values}")

# Calculate total combinations
import itertools
total = 1
for values in param_space.values():
    total *= len(values)
print(f"\nTotal combinations to test: {total}")

Parameter Space:
  short_window: [5, 10, 15, 20]
  long_window: [30, 40, 50, 60]
  shares: [10]

Total combinations to test: 16


In [17]:
# Run optimization
optimizer = GridSearchOptimizer(verbose=True)

optimization_results = optimizer.optimize(
    data=data,
    config=config,
    strategy_class=MovingAverageStrategy,
    param_space=param_space,
    metric='sharpe_ratio'  # Optimize for Sharpe ratio
)

print("\nOptimization completed!")

Testing 16 parameter combinations...


Grid Search: 100%|██████████| 16/16 [00:01<00:00, 12.24it/s]


Optimization completed!


In [18]:
# Display top 10 parameter combinations
print("\nTop 10 Parameter Combinations (by Sharpe Ratio):")
print("=" * 80)
top_10 = optimization_results.head(10)[['short_window', 'long_window', 'sharpe_ratio', 'total_return', 'max_drawdown', 'total_trades']]
print(top_10.to_string(index=False))


Top 10 Parameter Combinations (by Sharpe Ratio):
 short_window  long_window  sharpe_ratio  total_return  max_drawdown  total_trades
            5           30      0.277009     10.953612      3.620396            35
            5           40      0.139994      9.318646      3.447507            31
           20           40      0.078661      8.578330      4.475790            23
           15           30      0.076716      8.576281      3.865200            31
            5           50      0.042962      8.137525      3.524739            29
           20           30      0.038124      8.081468      4.353849            31
           10           50      0.017798      7.831042      3.796065            25
           10           30      0.007804      7.698038      4.583326            31
           15           40      0.002534      7.636113      5.422479            23
            5           60      0.000000      0.000000      0.000000             0


In [19]:
# Get best parameters
best_params = optimization_results.iloc[0]

print("\nBest Parameters:")
print(f"  Short Window: {int(best_params['short_window'])} days")
print(f"  Long Window: {int(best_params['long_window'])} days")
print(f"\nPerformance with Best Parameters:")
print(f"  Sharpe Ratio: {best_params['sharpe_ratio']:.2f}")
print(f"  Total Return: {best_params['total_return']:.2f}%")
print(f"  Max Drawdown: {best_params['max_drawdown']:.2f}%")
print(f"  Total Trades: {int(best_params['total_trades'])}")


Best Parameters:
  Short Window: 5 days
  Long Window: 30 days

Performance with Best Parameters:
  Sharpe Ratio: 0.28
  Total Return: 10.95%
  Max Drawdown: 3.62%
  Total Trades: 35


In [20]:
# Run backtest with optimized parameters
optimized_strategy = MovingAverageStrategy(
    short_window=int(best_params['short_window']),
    long_window=int(best_params['long_window']),
    shares=10,
    name="MA_Optimized"
)

backtest = Backtest(data=data, config=config)
optimized_results = backtest.run([optimized_strategy])

# Visualize
fig = plot_equity_curve(optimized_results)
fig.update_layout(title="Optimized Moving Average Strategy")
fig.show()

Running strategies: 100%|██████████| 1/1 [00:00<00:00, 13.63it/s]


In [21]:
# Visualize trading signals for optimized strategy
optimized_result = optimized_results.get_strategy("MA_Optimized")
fig = optimized_result.plot_trades(data)
fig.show()

## Summary

In this notebook, we covered:

1. ✅ **Data Loading**: Using yfinance to download OHLCV data
2. ✅ **Commission Setup**: Configuring percentage-based commissions
3. ✅ **Running Backtests**: Testing a Moving Average strategy
4. ✅ **Strategy Comparison**: Comparing MA, Buy & Hold, and DCA strategies
5. ✅ **Parameter Optimization**: Using GridSearch to find optimal MA parameters

### Key Takeaways:

- The framework is **asset-agnostic** - works with stocks, forex, crypto, etc.
- Multiple commission types are supported (percentage, flat, tiered)
- Strategies can be easily compared using built-in comparison methods
- Parameter optimization helps find the best strategy configuration
- The framework provides 20+ performance metrics automatically

### Next Steps:

- Explore candlestick pattern strategies
- Learn technical indicator strategies (RSI, MACD, Bollinger Bands)
- Create machine learning-based strategies
- Implement custom commission models
- Use advanced optimization techniques